In [2]:
import os
import numpy as np
import pandas as pd
import pickle
import time
import matplotlib.pyplot as plt
from pathlib import Path
from audiolib import AudioDataset, AudioFile

In [3]:
# Load data filters
filter_path = Path('../data/interim/train')
data_filters = os.listdir(filter_path)
data_filters.remove('.DS_Store')
print('Filter file names:', data_filters)
print('\nExample filter file:')
pd.read_csv(filter_path/data_filters[0]).head()

Filter file names: ['vocal_acoustic-filter.csv', 'bass_acoustic-filter.csv', 'vocal_electronic-filter.csv', 'string_electronic-filter.csv', 'brass_electronic-filter.csv', 'guitar_acoustic-filter.csv', 'string_acoustic-filter.csv', 'flute_acoustic-filter.csv', 'reed_acoustic-filter.csv', 'brass_acoustic-filter.csv', 'keyboard_acoustic-filter.csv', 'bass_electronic-filter.csv', 'guitar_electronic-filter.csv']

Example filter file:


,instrument,keep
0,vocal_acoustic_009,1
1,vocal_acoustic_028,1
2,vocal_acoustic_023,1
3,vocal_acoustic_024,1
4,vocal_acoustic_016,1


In [4]:
# Store filters in dict
instr_filtered = {}
for filter_file in data_filters:
    df_filter = pd.read_csv(filter_path/filter_file)
    instr_name = filter_file.split('-')[0]
    instr_filtered[instr_name] = (
        list(df_filter.loc[df_filter['keep'] == 1, 'instrument'].values))
print('Keys extracted from file names:', instr_filtered.keys())
print('\nExample of instruments to keep according to vocal acoustic csv file:',
      instr_filtered['vocal_acoustic'])

Keys extracted from file names: dict_keys(['vocal_acoustic', 'bass_acoustic', 'vocal_electronic', 'string_electronic', 'brass_electronic', 'guitar_acoustic', 'string_acoustic', 'flute_acoustic', 'reed_acoustic', 'brass_acoustic', 'keyboard_acoustic', 'bass_electronic', 'guitar_electronic'])

Example of instruments to keep according to vocal acoustic csv file: ['vocal_acoustic_009', 'vocal_acoustic_028', 'vocal_acoustic_023', 'vocal_acoustic_024', 'vocal_acoustic_016', 'vocal_acoustic_025', 'vocal_acoustic_026', 'vocal_acoustic_003', 'vocal_acoustic_002', 'vocal_acoustic_013', 'vocal_acoustic_015', 'vocal_acoustic_011', 'vocal_acoustic_014', 'vocal_acoustic_018', 'vocal_acoustic_012', 'vocal_acoustic_027', 'vocal_acoustic_019', 'vocal_acoustic_021', 'vocal_acoustic_008', 'vocal_acoustic_005', 'vocal_acoustic_022', 'vocal_acoustic_004', 'vocal_acoustic_020', 'vocal_acoustic_029', 'vocal_acoustic_001', 'vocal_acoustic_017', 'vocal_acoustic_006', 'vocal_acoustic_010', 'vocal_acoustic_007']

In [5]:
# Load dataset and view instruments
train = AudioDataset(path='../data/raw/nsynth-train/')
print('Dataset unique instruments:', train.unique_instruments)
print('\nVocal acoustic dataset file prefixes:',
      train.file_names_nested['vocal_acoustic'].keys())

Dataset unique instruments: ['guitar_acoustic', 'bass_synthetic', 'organ_electronic', 'guitar_electronic', 'keyboard_electronic', 'keyboard_acoustic', 'vocal_synthetic', 'string_acoustic', 'reed_acoustic', 'flute_acoustic', 'mallet_electronic', 'mallet_synthetic', 'brass_acoustic', 'guitar_synthetic', 'flute_synthetic', 'mallet_acoustic', 'synth_lead_synthetic', 'bass_electronic', 'keyboard_synthetic', 'vocal_acoustic', 'reed_synthetic', 'organ_acoustic', 'reed_electronic', 'vocal_electronic', 'bass_acoustic', 'string_electronic', 'brass_electronic', 'flute_electronic']

Vocal acoustic dataset file prefixes: dict_keys(['vocal_acoustic_009', 'vocal_acoustic_028', 'vocal_acoustic_023', 'vocal_acoustic_024', 'vocal_acoustic_016', 'vocal_acoustic_025', 'vocal_acoustic_026', 'vocal_acoustic_003', 'vocal_acoustic_002', 'vocal_acoustic_013', 'vocal_acoustic_015', 'vocal_acoustic_011', 'vocal_acoustic_014', 'vocal_acoustic_018', 'vocal_acoustic_012', 'vocal_acoustic_027', 'vocal_acoustic_019',

### Load audio

In [20]:
def save_spectrograms(dataset_path, instr_name, time_intervals=1, resolution=5, start=0, end=-1,
                      save_path='../data/processed/'):
    """
    Load instrument audio files and save their spectrograms
    """
    # Fixed values
    dataset = AudioDataset(dataset_path)
    instr_ids = instr_filtered[instr_name]
    num_files = sum([len(dataset.file_names_nested[instr_name][id]) for id in instr_ids])
    # Dynamically updated values
    success_count = 0
    fail_count = 0
    # Save path
    dataset_name = str(dataset_path).split('/')[-1]
    save_path = Path(os.path.join(
        save_path,
        dataset_name,
        f'time_intervals={time_intervals}',
        f'resolution={resolution}',
        instr_name
    ))
    print('Generating {} {} spectrograms'.format(num_files, instr_name))
    for instr_id in instr_ids:
        instr_save_path = save_path/instr_id
        os.makedirs(instr_save_path, exist_ok=True)
        for file_name in dataset.file_names_nested[instr_name][instr_id]:
            print('- File {} - {} successes / {} failures'.format(
                     file_name, success_count, fail_count),
                  end ='\r')
            try:
                af = dataset.load_file(file_name)
                if 'fast_decay' not in af.info['qualities_str']:
                    spec = (af.audio
                            .trim(start, end)
                            .to_spectrogram(time_intervals, resolution, cqt=True))
                    with open(instr_save_path/(file_name+'.spec'), 'wb') as f:
                        pickle.dump(spec, f)
                    success_count += 1
            except Exception as exc:
                #print(exc)
                fail_count += 1
                
    print('\nFinished. Saved spectrograms to {}\n'.format(save_path))

In [21]:
for instr in list(instr_filtered.keys()):
    save_spectrograms('../data/raw/nsynth-train/', instr, start=0.2, end=3)

Generating 3820 vocal_acoustic spectrograms
- File vocal_acoustic_007-043-075 - 3572 successes / 247 failures
Finished. Saved spectrograms to ../data/processed/time_intervals=1/resolution=5/vocal_acoustic

Generating 200 bass_acoustic spectrograms
- File bass_acoustic_000-060-025 - 157 successes / 24 failures
Finished. Saved spectrograms to ../data/processed/time_intervals=1/resolution=5/bass_acoustic

Generating 140 vocal_electronic spectrograms
- File vocal_electronic_002-057-127 - 135 successes / 4 failures
Finished. Saved spectrograms to ../data/processed/time_intervals=1/resolution=5/vocal_electronic

Generating 84 string_electronic spectrograms
- File string_electronic_000-080-127 - 79 successes / 4 failures
Finished. Saved spectrograms to ../data/processed/time_intervals=1/resolution=5/string_electronic

Generating 70 brass_electronic spectrograms
- File brass_electronic_001-076-127 - 66 successes / 3 failures
Finished. Saved spectrograms to ../data/processed/time_intervals=1/re